# Python Section 2

## Question 9: Distance Matrix Calculation

Create a function named calculate_distance_matrix that takes the dataset-2.csv as input and generates a DataFrame representing distances between IDs.

The resulting DataFrame should have cumulative distances along known routes, with diagonal values set to 0. If distances between toll locations A to B and B to C are known, then the distance from A to C should be the sum of these distances. Ensure the matrix is symmetric, accounting for bidirectional distances between toll locations (i.e. A to B is equal to B to A).

In [1]:
import pandas as pd

# Specify the path to your CSV file
file_path ='C:/Users/Nitesh Sharma/Submission/dataset-2.csv'

# Load the dataset
df = pd.read_csv(file_path)

# Create a unique list of IDs from both 'id_start' and 'id_end'
unique_ids = pd.unique(df[['id_start', 'id_end']].values.ravel('K'))

# Initialize the distance matrix with NaN
distance_matrix = pd.DataFrame(index=unique_ids, columns=unique_ids)
distance_matrix = distance_matrix.astype(float)  # Set data type to float

# Fill the distance matrix
for _, row in df.iterrows():
    id_a = row['id_start']
    id_b = row['id_end']
    distance = float(row['distance'])  # Ensure distance is a float
    distance_matrix.at[id_a, id_b] = distance
    distance_matrix.at[id_b, id_a] = distance  # Assuming the distance is symmetric

# To fill NaN values with 0 or any other value if required:
distance_matrix.fillna(0, inplace=True)

# Print the resulting distance matrix
print(distance_matrix)


         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400      0.0      9.7      0.0      0.0      0.0      0.0      0.0   
1001402      9.7      0.0     20.2      0.0      0.0      0.0      0.0   
1001404      0.0     20.2      0.0     16.0      0.0      0.0      0.0   
1001406      0.0      0.0     16.0      0.0     21.7      0.0      0.0   
1001408      0.0      0.0      0.0     21.7      0.0     11.1      0.0   
1001410      0.0      0.0      0.0      0.0     11.1      0.0     15.6   
1001412      0.0      0.0      0.0      0.0      0.0     15.6      0.0   
1001414      0.0      0.0      0.0      0.0      0.0      0.0     18.2   
1001416      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001418      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001420      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001422      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001424      0.0      0.0      0.0    

## Question 10: Unroll Distance Matrix

Create a function unroll_distance_matrix that takes the DataFrame created in Question 9. The resulting DataFrame should have three columns: columns id_start, id_end, and distance.

All the combinations except for same id_start to id_end must be present in the rows with their distance values from the input DataFrame.

In [2]:
import pandas as pd

# Load the distance matrix from a CSV file
file_path = 'C:/Users/Nitesh Sharma/Submission/dataset-2.csv'  # Replace with your actual file path
distance_matrix = pd.read_csv(file_path, index_col=0)

# Ensure that the distance matrix has numeric values
distance_matrix = distance_matrix.apply(pd.to_numeric, errors='coerce')

def unroll_distance_matrix(distance_matrix):
    # Print the structure of the DataFrame for debugging
    print("Distance Matrix:")
    print(distance_matrix)

    # Initialize a list to hold the unrolled data
    unrolled_data = []

    # Get the list of valid IDs from the index and columns
    valid_ids = distance_matrix.index.tolist()
    
    print("Valid IDs:")
    print(valid_ids)

    # Iterate through the matrix and extract non-diagonal elements
    for id_start in valid_ids:
        for id_end in valid_ids:
            if id_start != id_end:  # Exclude same id_start and id_end
                try:
                    distance = distance_matrix.at[id_start, id_end]

                    # Check if the distance is a valid number
                    if isinstance(distance, (int, float)) and pd.notna(distance):
                        unrolled_data.append({'id_start': id_start, 'id_end': id_end, 'distance': distance})
                except KeyError:
                    print(f"KeyError: Could not find distance for {id_start}, {id_end}")

    # Create a DataFrame from the unrolled data
    unrolled_df = pd.DataFrame(unrolled_data)
    return unrolled_df

# Call the function and print the output
unrolled_df = unroll_distance_matrix(distance_matrix)
print("Unrolled Distance DataFrame:")
print(unrolled_df)


Distance Matrix:
           id_end  distance
id_start                   
1001400   1001402       9.7
1001402   1001404      20.2
1001404   1001406      16.0
1001406   1001408      21.7
1001408   1001410      11.1
1001410   1001412      15.6
1001412   1001414      18.2
1001414   1001416      13.2
1001416   1001418      13.6
1001418   1001420      12.9
1001420   1001422       9.6
1001422   1001424      11.4
1001424   1001426      18.6
1001426   1001428      15.8
1001428   1001430       8.6
1001430   1001432       9.0
1001432   1001434       7.9
1001434   1001436       4.0
1001436   1001438       9.0
1001436   1001437       5.0
1001438   1001437       4.0
1001438   1001440      10.0
1001440   1001442       3.9
1001442   1001488       4.5
1001488   1004356       4.0
1004356   1004354       2.0
1004354   1004355       2.0
1004355   1001444       0.7
1001444   1001446       6.6
1001446   1001448       9.6
1001448   1001450      15.7
1001450   1001452       9.9
1001452   1001454      11.3
100

## Question 11: Finding IDs within Percentage Threshold

Create a function find_ids_within_ten_percentage_threshold that takes the DataFrame created in Question 10 and a reference value from the id_start column as an integer.

Calculate average distance for the reference value given as an input and return a sorted list of values from id_start column which lie within 10% (including ceiling and floor) of the reference value's average.

In [3]:
import pandas as pd

# Provide the file path to your CSV file here
file_path ='C:/Users/Nitesh Sharma/Submission/dataset-2.csv'  # Replace with your actual file path

# Read the CSV file into a pandas DataFrame
unrolled_df = pd.read_csv(file_path)

def find_ids_within_ten_percentage_threshold(unrolled_df, reference_id):
    # Debugging: Print the columns of the DataFrame
    print("Columns in unrolled_df:", unrolled_df.columns.tolist())

    # Ensure the reference ID exists in the DataFrame
    if reference_id not in unrolled_df['id_start'].values:
        raise ValueError(f"Reference ID {reference_id} not found in id_start column.")
    
    # Calculate the average distance for the reference ID
    avg_distance = unrolled_df[unrolled_df['id_start'] == reference_id]['distance'].mean()
    
    # Calculate the lower and upper thresholds (10% of avg_distance)
    lower_threshold = avg_distance * 0.9
    upper_threshold = avg_distance * 1.1
    
    # Find IDs within the threshold
    ids_within_threshold = unrolled_df[
        (unrolled_df['distance'] >= lower_threshold) & 
        (unrolled_df['distance'] <= upper_threshold)
    ]['id_start'].unique()
    
    # Sort the results
    sorted_ids = sorted(ids_within_threshold)
    
    return sorted_ids

# Call the function with your DataFrame and a reference ID
reference_id = 1001400  # Replace with your actual reference ID
try:
    result_ids = find_ids_within_ten_percentage_threshold(unrolled_df, reference_id)
    print(f"IDs within 10% of the average distance for {reference_id}: {result_ids}")
except ValueError as e:
    print(e)


Columns in unrolled_df: ['id_start', 'id_end', 'distance']
IDs within 10% of the average distance for 1001400: [1001400, 1001420, 1001430, 1001436, 1001438, 1001446, 1001450, 1001456, 1001468]


## Question 12: Calculate Toll Rate

Create a function calculate_toll_rate that takes the DataFrame created in Question 10 as input and calculates toll rates based on vehicle types.

The resulting DataFrame should add 5 columns to the input DataFrame: moto, car, rv, bus, and truck with their respective rate coefficients. The toll rates should be calculated by multiplying the distance with the given rate coefficients for each vehicle type:

1. 0.8 for moto
2. 1.2 for car
3. 1.5 for rv
4. 2.2 for bus
5. 3.6 for truck

In [4]:
import pandas as pd

def calculate_toll_rate(unrolled_df):
    # Debugging: Ensure 'distance' column exists
    if 'distance' not in unrolled_df.columns:
        raise KeyError("'distance' column not found in the DataFrame.")
    
    # Calculate toll rates for each vehicle type by multiplying distance with the respective rate coefficients
    unrolled_df['moto'] = unrolled_df['distance'] * 0.8  # Motorcycle toll rate
    unrolled_df['car'] = unrolled_df['distance'] * 1.2   # Car toll rate
    unrolled_df['rv'] = unrolled_df['distance'] * 1.5    # RV toll rate
    unrolled_df['bus'] = unrolled_df['distance'] * 2.2   # Bus toll rate
    unrolled_df['truck'] = unrolled_df['distance'] * 3.6 # Truck toll rate

    return unrolled_df

# Provide the file path to your CSV file here
file_path = 'C:/Users/Nitesh Sharma/Submission/dataset-2.csv'

# Read the CSV file into a pandas DataFrame
unrolled_df = pd.read_csv(file_path)

# Call the function to calculate toll rates
updated_df = calculate_toll_rate(unrolled_df)

# Print the DataFrame with the added toll rate columns
print(updated_df.head())


   id_start   id_end  distance   moto    car     rv    bus  truck
0   1001400  1001402       9.7   7.76  11.64  14.55  21.34  34.92
1   1001402  1001404      20.2  16.16  24.24  30.30  44.44  72.72
2   1001404  1001406      16.0  12.80  19.20  24.00  35.20  57.60
3   1001406  1001408      21.7  17.36  26.04  32.55  47.74  78.12
4   1001408  1001410      11.1   8.88  13.32  16.65  24.42  39.96


## Question 13: Calculate Time-Based Toll Rates

Create a function named calculate_time_based_toll_rates that takes the DataFrame created in Question 12 as input and calculates toll rates for different time intervals within a day.

The resulting DataFrame should have these five columns added to the input: start_day, start_time, end_day, and end_time.

1. start_day, end_day must be strings with day values (from Monday to Sunday in proper case)
2. start_time and end_time must be of type datetime.time() with the values from time range given below.

Modify the values of vehicle columns according to the following time ranges:

Weekdays (Monday - Friday):

1. From 00:00:00 to 10:00:00: Apply a discount factor of 0.8
2. From 10:00:00 to 18:00:00: Apply a discount factor of 1.2
3. From 18:00:00 to 23:59:59: Apply a discount factor of 0.8

Weekends (Saturday and Sunday):

1. Apply a constant discount factor of 0.7 for all times.

For each unique (id_start, id_end) pair, cover a full 24-hour period (from 12:00:00 AM to 11:59:59 PM) and span all 7 days of the week (from Monday to Sunday).

In [5]:
import pandas as pd

# Function to calculate toll rates based on vehicle type
def calculate_toll_rate(df):
    # Add columns for different vehicle types with rate coefficients
    df['moto'] = df['distance'] * 0.8
    df['car'] = df['distance'] * 1.2
    df['rv'] = df['distance'] * 1.5
    df['bus'] = df['distance'] * 2.2
    df['truck'] = df['distance'] * 3.6
    return df

# Load the data
file_path = 'C:/Users/Nitesh Sharma/Submission/dataset-2.csv'
df = pd.read_csv(file_path)

# Apply the toll rate calculation
df = calculate_toll_rate(df)

# Save the updated DataFrame with toll rates
df.to_csv('C:/Users/Nitesh Sharma/Documents/Captured_Images/unrolled_data_with_toll_rates.csv', index=False)

print("Toll rate columns added successfully.")


Toll rate columns added successfully.


In [6]:
import pandas as pd
import datetime

# Function to calculate time-based toll rates
def calculate_time_based_toll_rates(df):
    # Weekday and weekend lists
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    weekends = ['Saturday', 'Sunday']

    # Create an empty list to hold the new rows
    new_rows = []

    # Loop over each (id_start, id_end) pair
    for idx, row in df.iterrows():
        # For all days of the week
        for day in weekdays + weekends:
            # Generate three time intervals
            for start_time, end_time, discount in [
                (datetime.time(0, 0, 0), datetime.time(10, 0, 0), 0.8),    # 00:00:00 to 10:00:00
                (datetime.time(10, 0, 0), datetime.time(18, 0, 0), 1.2),   # 10:00:00 to 18:00:00
                (datetime.time(18, 0, 0), datetime.time(23, 59, 59), 0.8)  # 18:00:00 to 23:59:59
            ]:
                new_row = row.copy()
                new_row['start_day'] = day
                new_row['start_time'] = start_time
                new_row['end_day'] = day
                new_row['end_time'] = end_time

                # Apply weekday discount factors
                if day in weekdays:
                    new_row['moto'] *= discount
                    new_row['car'] *= discount
                    new_row['rv'] *= discount
                    new_row['bus'] *= discount
                    new_row['truck'] *= discount
                # Apply weekend discount factor (constant 0.7)
                elif day in weekends:
                    new_row['moto'] *= 0.7
                    new_row['car'] *= 0.7
                    new_row['rv'] *= 0.7
                    new_row['bus'] *= 0.7
                    new_row['truck'] *= 0.7

                # Add the new row to the list
                new_rows.append(new_row)

    # Convert the list of rows into a DataFrame
    time_based_df = pd.concat([pd.DataFrame([row]) for row in new_rows], ignore_index=True)

    # Reorder the columns as per the specified format
    column_order = ['id_start', 'id_end', 'distance', 'start_day', 'start_time', 'end_day', 'end_time', 'moto', 'car', 'rv', 'bus', 'truck']
    time_based_df = time_based_df[column_order]

    return time_based_df

# Load the DataFrame with toll rates
file_path = 'C:/Users/Nitesh Sharma/Documents/Captured_Images/unrolled_data_with_toll_rates.csv'
df = pd.read_csv(file_path)

# Apply the time-based toll rate calculation
time_based_toll_df = calculate_time_based_toll_rates(df)

# Save the resulting DataFrame to a new file
output_file_path = 'C:/Users/Nitesh Sharma/Documents/Captured_Images/unrolled_data_with_time_based_toll_rates.csv'
time_based_toll_df.to_csv(output_file_path, index=False)

# Print the first 5 rows of the resulting DataFrame to the console
print(time_based_toll_df.head())

print("\nTime-based toll rates calculated, printed, and saved successfully.")


    id_start     id_end  distance start_day start_time  end_day  end_time  \
0  1001400.0  1001402.0       9.7    Monday   00:00:00   Monday  10:00:00   
1  1001400.0  1001402.0       9.7    Monday   10:00:00   Monday  18:00:00   
2  1001400.0  1001402.0       9.7    Monday   18:00:00   Monday  23:59:59   
3  1001400.0  1001402.0       9.7   Tuesday   00:00:00  Tuesday  10:00:00   
4  1001400.0  1001402.0       9.7   Tuesday   10:00:00  Tuesday  18:00:00   

    moto     car     rv     bus   truck  
0  6.208   9.312  11.64  17.072  27.936  
1  9.312  13.968  17.46  25.608  41.904  
2  6.208   9.312  11.64  17.072  27.936  
3  6.208   9.312  11.64  17.072  27.936  
4  9.312  13.968  17.46  25.608  41.904  

Time-based toll rates calculated, printed, and saved successfully.
